In [18]:
import sqlite3

# Establish a connection to the SQLite database
connection = sqlite3.connect("pets.db")
cursor = connection.cursor()
print("Database connected successfully!")


Database connected successfully!


In [19]:
# Create the 'kind' table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS kind (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type TEXT NOT NULL UNIQUE
    )
""")

# Create the 'pet' table with a foreign key reference to 'kind'
cursor.execute("""
    CREATE TABLE IF NOT EXISTS pet (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        kind_id INTEGER NOT NULL,
        age INTEGER,
        owner TEXT,
        FOREIGN KEY (kind_id) REFERENCES kind(id)
    )
""")

# Commit the changes
connection.commit()
print("Tables created successfully!")


Tables created successfully!


In [20]:
# Insert pet types into the 'kind' table
pet_types = [("dog",), ("cat",)]
cursor.executemany("INSERT OR IGNORE INTO kind (type) VALUES (?)", pet_types)

# Commit the changes
connection.commit()
print("Pet types inserted into 'kind' table!")


Pet types inserted into 'kind' table!


In [21]:
# Fetch kind IDs for reference
cursor.execute("SELECT id, type FROM kind")
kind_map = {kind: kind_id for kind_id, kind in cursor.fetchall()}

# Insert pet records with foreign key references
pets = [
    ("dorothy", kind_map["dog"], 9, "greg"),
    ("suzy", kind_map["dog"], 9, "greg"),
    ("casey", kind_map["dog"], 9, "greg"),
    ("heidi", kind_map["dog"], 9, "greg"),
    ("sandy", kind_map["cat"], 9, "greg"),
    ("muffin", kind_map["cat"], 9, "greg"),
    ("shadow", kind_map["cat"], 9, "greg"),
]

cursor.executemany("""
    INSERT INTO pet (name, kind_id, age, owner) 
    VALUES (?, ?, ?, ?)
""", pets)

# Commit the changes
connection.commit()
print("Pets inserted into 'pet' table!")


Pets inserted into 'pet' table!


In [22]:
# Select all dogs using a JOIN
cursor.execute("""
    SELECT pet.id, pet.name, kind.type, pet.age, pet.owner 
    FROM pet 
    JOIN kind ON pet.kind_id = kind.id 
    WHERE kind.type = ?
""", ("dog",))

# Fetch and display results
rows = cursor.fetchall()
print("Dogs in the database:")
for row in rows:
    print(row)


Dogs in the database:
(1, 'dorothy', 'dog', 9, 'greg')
(2, 'suzy', 'dog', 9, 'greg')
(3, 'casey', 'dog', 9, 'greg')
(4, 'heidi', 'dog', 9, 'greg')
(8, 'dorothy', 'dog', 9, 'greg')
(9, 'suzy', 'dog', 9, 'greg')
(10, 'casey', 'dog', 9, 'greg')
(11, 'heidi', 'dog', 9, 'greg')
(15, 'dorothy', 'dog', 9, 'greg')
(16, 'suzy', 'dog', 9, 'greg')
(17, 'casey', 'dog', 9, 'greg')
(18, 'heidi', 'dog', 9, 'greg')


In [23]:
# Update age of 'sandy'
cursor.execute("""
    UPDATE pet 
    SET age = ? 
    WHERE name = ?
""", (11, "sandy"))

# Commit the changes
connection.commit()
print("Updated 'sandy' age to 11!")

# Verify the update
cursor.execute("SELECT * FROM pet WHERE name = ?", ("sandy",))
print("Updated record:", cursor.fetchone())


Updated 'sandy' age to 11!
Updated record: (19, 'sandy', 2, 11, 'greg')


In [24]:
# Delete the first record in the 'pet' table based on the lowest ID
cursor.execute("""
    DELETE FROM pet 
    WHERE id = (SELECT MIN(id) FROM pet)
""")

# Commit the changes
connection.commit()
print("Deleted the first record in the 'pet' table!")

# Verify deletion
cursor.execute("SELECT * FROM pet")
remaining_rows = cursor.fetchall()
print("Remaining records in 'pet' table:")
for row in remaining_rows:
    print(row)


Deleted the first record in the 'pet' table!
Remaining records in 'pet' table:
(2, 'suzy', 1, 9, 'greg')
(3, 'casey', 1, 9, 'greg')
(4, 'heidi', 1, 9, 'greg')
(8, 'dorothy', 1, 9, 'greg')
(9, 'suzy', 1, 9, 'greg')
(10, 'casey', 1, 9, 'greg')
(11, 'heidi', 1, 9, 'greg')
(15, 'dorothy', 1, 9, 'greg')
(16, 'suzy', 1, 9, 'greg')
(17, 'casey', 1, 9, 'greg')
(18, 'heidi', 1, 9, 'greg')
(19, 'sandy', 2, 11, 'greg')
(20, 'muffin', 2, 9, 'greg')
(21, 'shadow', 2, 9, 'greg')


In [25]:
# Close the database connection
connection.close()
print("Database connection closed!")


Database connection closed!
